# Air Sac Tracking Prototype code
Lara S. Burchardt & Wim Pouw

lara.burchardt@donders.ru.nl

Next to body movements and acoustics, we would also like to track the air sac's inflation of the Siamangs. The air sac naturally forms a spherical(3D) or circular (2D) shape, and such shapes are retrievable from an image using the hough transform, and a bit of pre-processing of the images to get the optimal representation of the relevent edges of the air sac.

This code takes as input a sample video with a close up of a Siamang, and then tracks the air sac when it takes a sufficiently circular shape. The result is shown below; it is not perfect, but with a bit of smoothing this can function as a good air sac tracker. This code is very much under development, there are many ways to improve further.

In [5]:
# import the necessary packages
import numpy as np
import argparse
import cv2
import pandas as pd
from skimage import io, feature, color, measure, draw, img_as_float
import numpy as np
import csv
import random2
import statistics
import scipy
from scipy import signal
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import itertools
import os
from os import listdir
from os.path import isfile, join


#resused code from 
#https://pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
#https://stackoverflow.com/questions/31705355/how-to-detect-circlular-region-in-images-and-centre-it-with-python

## Define Prepocessing Function

In [6]:
#define parameters for HoughTransform outside of function to be able to save and manipulate easier
def hougdraw(submitted_image, dp = 1, mindist = 10000, param1 = 10, param2=22, minradius = 5, maxradius=250):
    circles = cv2.HoughCircles(submitted_image, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist = mindist,  
                               param1 = param1, param2 = param2, 
                               minRadius = minradius, maxRadius = maxradius)
    if circles is not None:
        circles = np.round(circles[0, 0:1]).astype("int")
        circle1 = circles[0,0]
        circle2 = circles[0,1]
        circle3 = circles[0,2]
        for(x, y, r) in circles:
            cv2.circle(submitted_image, (x, y), r, (255, 255, 0), 2) #version without drawing roi back on whole image 
    return(submitted_image)
    
# define function for preprocessing
def preprocessing(image, phase1_medianblur = 27, 
                              phase2_dilate = 7, 
                              phase2_medianblur = 19, 
                              alpha = 2, 
                              beta= 30,
                              thresh_div_1=10,
                              thresh_div_2=15):
    #image0 = hougdraw(image)
    #convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #brightness change
    gray = cv2.convertScaleAbs(gray, alpha = alpha, beta = beta)
    #set dynamic tresholds for canny (and thus also for hough)
    mean_intensity = np.median(gray)
    threshold1 = int(max(0, (1.0 - 0.33) * mean_intensity/thresh_div_1))
    threshold2 = int(min(255, (1.0 + 0.33) * mean_intensity/thresh_div_2))    
    image1_h = hougdraw(gray,param1= threshold1, param2 =threshold2)
    #blur
    image2 = cv2.medianBlur(gray, phase1_medianblur)
    image2_h = hougdraw(image2.copy(),param1= threshold1, param2 =threshold2)
    #dynamic thresholds for canny edge detection based on intensity of image
    #Thresholds one standard deviation above and below median intensity
    #edge detection
    image3 = cv2.Canny(image2, threshold1, threshold2)
    image3_h = hougdraw(image3.copy(), param1= threshold1, param2 =threshold2)
    #dilation and second blur
    submitted = cv2.dilate(image3, None, iterations= phase2_dilate)  
    image4 = cv2.medianBlur(submitted, phase2_medianblur) 
    image4_h = hougdraw(image4.copy(), param1= threshold1, param2 =threshold2)
    #add hough
    return image1_h, image2_h, image3_h, image4_h
############################################

def preprocessing2(image, phase1_medianblur = 27, 
                              phase2_dilate = 5, 
                              phase2_medianblur = 27, 
                              alpha = 2, 
                              beta= 30,
                              param1canny=10,
                              param2canny=15):
    #image0 = hougdraw(image)
    #convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #brightness change
    gray = cv2.convertScaleAbs(gray, alpha = alpha, beta = beta)
    #set dynamic tresholds for canny (and thus also for hough)
    mean_intensity = np.median(gray)
    threshold1 = int(max(0, (1.0 - 0.33) * mean_intensity/param1canny))
    threshold2 = int(min(255, (1.0 + 0.33) * mean_intensity/param2canny))    
    #blur
    image2 = cv2.medianBlur(gray, phase1_medianblur)
    #dynamic thresholds for canny edge detection based on intensity of image
    #Thresholds one standard deviation above and below median intensity
    #edge detection
    image3 = cv2.Canny(image2, threshold1, threshold2)
    #dilation and second blur
    submitted = cv2.dilate(image3, None, iterations= phase2_dilate)  
    image4 = cv2.medianBlur(submitted, phase2_medianblur) 
    #add hough
    image4 = np.float32(image4)
    return image4, threshold1, threshold2

def matriximages(outputname, wimagenumber, himagenumber, imagelist, marginbetweenimages, vertnames, hornames):
    """
    matrix images takes in the full path name of the outputfolder for storing the images
    it takes as input a list of full path names of the images
    the height is given in image numbers and same for width
    the vert names is a list of column names
    the hor names is a list of row names
    """
    margin = marginbetweenimages
    #img = imagelist  
    w = wimagenumber
    h = himagenumber
    n = w*h

    #imgs = [cv2.imread(i) for i in imagelist]
    imgs = imagelist
    #if any(i.shape != imgs[0].shape for i in imgs[1:]):
    #    raise ValueError('Not all images have the same shape.')

    img_h, img_w = imgs[0].shape#img_h, img_w, img_c = imgs[0].shape

    m_x = 0
    m_y = 0
    if marginbetweenimages is not None:
        margin = marginbetweenimages
        m_x = int(margin*img_w)
        m_y = int(margin*img_h)
        
    imgmatrix = np.zeros((img_h * h + m_y * (h - 1),
                          img_w * w + m_x * (w - 1)),
                         np.uint8)

    imgmatrix.fill(255)    

    positions = itertools.product(range(w), range(h))
    for (x_i, y_i), img in zip(positions, imgs):
        x = x_i * (img_w + m_x)
        y = y_i * (img_h + m_y)
        imgmatrix[y:y+img_h, x:x+img_w] = img #imgmatrix[y:y+img_h, x:x+img_w, :] = img
    #add text
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(imgmatrix, hornames[0], (0*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[1], (1*img_w,  int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[2], (2*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[3], (3*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[4], (4*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[0], (0, 1*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[1], (0, 2*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[2], (0, 3*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[3], (0, 4*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    #cv2.putText(imgmatrix, vertnames[4], (0, 5*img_h), font, 10, (0, 255, 0), 5, cv2.LINE_AA)
    cv2.imwrite(outputname, imgmatrix)   
    
    print('done, look in your folder: '+ outputname)
    
    
def roidefinition(video, height_buffer, width_buffer, dp, minRadius, maxRadius):
    cap = cv2.VideoCapture(video)
    totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    #frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    #frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    #fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second
    #set up empty output dataframe
    column_names = ['x','y', 'r', 'frame'] # we are going to collect hough generated x,y,r,frame from random set of frames
    df_round_1 = pd.DataFrame(columns = column_names) 
    vector = range(0, int(totalFrames), 1)
    samp = random2.sample(vector, 45) #50 is the setting
    for rand in samp:
        # set frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES,rand)
        ret, frame = cap.read()
        ############################detect circles   
        output=frame.copy()
        # transform to grayscale image only using the roi part of the image
        image4, param1, param2 = preprocessing2(image=output)
        final_im = cv2.normalize(src=image4, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        #final_im = image4
        circles = cv2.HoughCircles(final_im, cv2.HOUGH_GRADIENT, 
                                   dp = dp,minDist = minDist,  
                                   param1 = param1,param2 = param2, 
                                   minRadius = minRadius, maxRadius = maxRadius)
        #if circles is not None:
        #    circles = np.round(circles[0, 0:1]).astype("int")
        if circles is not None:
            #circles = np.round(circles[0, 0:1]).astype("int")
            if circles is not None:
                circles = np.round(circles[0, 0:1]).astype("int")
                circle1 = circles[0,0]
                circle2 = circles[0,1]
                circle3 = circles[0,2]
            #save it to a row
            if circles is None:
                circle1 = "NA"
                circle2 = "NA"
                circle3 = "NA"
        new_row = [circles[0,0], circles[0,1],circles [0,2], rand]
        df_round_1.loc[len(df_round_1)] = new_row

    #when collected take some info
    median_x = df_round_1['x'].median() #order in pos : same as in original dataframe, so x,y, r
    median_y = df_round_1['y'].median()
    max_r = df_round_1['r'].max()
    min_r = df_round_1['r'].min()
    # order of parameters saved as roi with cv2.selectROI [Top_Left_X, Top_Left_Y, Width, Height]

    # explanation: height_1 is the maximum radius detected plus 30 pixels, that is used to determine the position of the 
    # upper left corner of the roi, same goes for width_1, both are currently the same, but that could change, so they are both
    # coded independently
    height_1 = max_r + height_buffer #i.e.100, in pixels
    width_1 = max_r + width_buffer #250, in pixels
    pos_x = median_x - width_1
    pos_y = median_y - height_1  
    # to be consistent with the roi nomenclature, we then calculate the width/height of the whole roi which is width_1 *2
        # all those values will then be used to crop picture in next round of tracking
    width_2 = width_1 * 2
    height_2 = height_1 *2
    #
    roi = [pos_x, pos_y, width_2, height_2]
    cap.release() #release the video
    return roi

## Main Function

This function is also used in the looper: process_all_videos.py

The tracker is run on the full video, it first defines an ROI defined in the first step. 
The output is saved in a table format and as a video with the detected circles drawn on top. 

In [ ]:
# preprocessing, choosing a roi on the tracking results of 50 random samples from input file
#https://github.com/patchy631/machine-learning/blob/main/computer_vision/cv2_edge_detection.ipynb

############settings that worked: c1_5_c2_10_al_1_b_12_dil_7_blur_27
alpha = 2
beta = 20
dp = 1
dilation = 5
phase1_medianblur = 27
#cannyt1 = 5
#cannyt2 = 12 
minDist = 10000
minRadius = 5 #minus 2times std
maxRadius = 260 #dynamic? 
###################
videofolder= '../Video/'
nameforfiles = 'June16_02'
videofilename = nameforfiles +'.mp4'
# Opens the Video file
cap = cv2.VideoCapture(videofolder+videofilename)
totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second

#set up empty output dataframe
column_names = ['x','y', 'r', 'frame'] # we are going to collect hough generated x,y,r,frame from random set of frames
df_round_1 = pd.DataFrame(columns = column_names) 
vector = range(0, int(totalFrames), 1)
samp = random2.sample(vector, 50) #50 is the setting

for rand in samp:
    # set frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES,rand)
    ret, frame = cap.read()
    ############################detect circles   
    output=frame.copy()
    # transform to grayscale image only using the roi part of the image
    image4, param1, param2 = preprocessing2(image=output)
    final_im = cv2.normalize(src=image4, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    circles = cv2.HoughCircles(final_im, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist = minDist,  
                               param1 = param1,param2 = param2, 
                               minRadius = minRadius, maxRadius = maxRadius)
    #if circles is not None:
    #    circles = np.round(circles[0, 0:1]).astype("int")
    if circles is not None:
        #circles = np.round(circles[0, 0:1]).astype("int")
        if circles is not None:
            circles = np.round(circles[0, 0:1]).astype("int")
            circle1 = circles[0,0]
            circle2 = circles[0,1]
            circle3 = circles[0,2]
        #save it to a row
        if circles is None:
            circle1 = "NA"
            circle2 = "NA"
            circle3 = "NA"
    new_row = [circles[0,0], circles[0,1],circles [0,2], rand]
    df_round_1.loc[len(df_round_1)] = new_row
    cv2.waitKey(1)
cv2.destroyAllWindows()

#when collected take some info
median_x = df_round_1['x'].median() #order in pos : same as in original dataframe, so x,y, r
median_y = df_round_1['y'].median()
max_r = df_round_1['r'].max()
min_r = df_round_1['r'].min()
    # order of parameters saved as roi with cv2.selectROI [Top_Left_X, Top_Left_Y, Width, Height]
   
    # explanation: height_1 is the maximum radius detected plus 30 pixels, that is used to determine the position of the 
    # upper left corner of the roi, same goes for width_1, both are currently the same, but that could change, so they are both
    # coded independently
height_1 = max_r + 100
width_1 = max_r + 250
pos_x = median_x - width_1
pos_y = median_y - height_1  
# to be consistent with the roi nomenclature, we then calculate the width/height of the whole roi which is width_1 *2
    # all those values will then be used to crop picture in next round of tracking
width_2 = width_1 * 2
height_2 = height_1 *2
#
roi = [pos_x, pos_y, width_2, height_2]

# cleaning up
#cap.set(cv2.CAP_PROP_POS_FRAMES,samp[1])
#ret, frame = cap.read()
#cv2.rectangle(frame,(int(roi[0]), int(roi[1])), (int(roi[0]+roi[2]), int(roi[1]+roi[3])), (0,0,0), 2) 
#cv2.imshow("output", frame) #ROI)
cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

################################

# Opens the Video file
cap = cv2.VideoCapture(videofolder+videofilename)
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second

#output video
out = cv2.VideoWriter('./Output/videos/output_'+videofilename,cv2.VideoWriter_fourcc(*'MP4V'), fps, 
                      (int(frameWidth), int(frameHeight)))
#set up empty output dataframe
column_names = ['frame','roi_x1','roi_y1','roi_x2', 'roi_y2', # info on region of interest for repetability
                'x','y', 'r', 'inputvideo','input_id',                    # actual results 
                'alpha', 'beta',                              # values of brightness and contrast pre processing
                'dp', 'minDist', 'param1', 'param2', 'minRadius',' maxRadius'] # parameters of hough circle transform 
df = pd.DataFrame(columns = column_names)
print("parameters set")
#initialize iterator
i=0
#ROI as automatically detected beforehand
pos_x =0 
pos_y= 0
width_2=0
height_2=0

#main loop                      
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
  ############################detect circles   
    output=frame.copy()
    print("preprocessing image")
    image4, param1, param2 = preprocessing2(image=output)
    print("done")
    #track demicircles 
    name = 'framenr_' + str(i+1) + '_framevid_' + videofilename[0:-4] 
    # original values: dp = 1, minDist = 10000, param1=1, param2=10, minRadius = 5, maxRadius= 250
    #submit to 
    image4 = image4[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]
    
    final_im = cv2.normalize(src=image4, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) 
    circles = cv2.HoughCircles(final_im, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist = minDist,  
                               param1 = param1,param2 = param2, minRadius = minRadius, maxRadius = maxRadius)   
    #cv2.rectangle(image, start_point, end_point, color, thickness)    
    cv2.rectangle(frame,(int(roi[0]), int(roi[1])), (int(roi[0]+roi[2]), int(roi[1]+roi[3])), (0,0,0), 2)    

    if circles is not None:
        #circles = np.round(circles[0, 0:1]).astype("int")
        if circles is not None:
            circles = np.round(circles[0, 0:1]).astype("int")
            circle1 = circles[0,0]+roi[0] #x  + plus the shift from the roi
            circle2 = circles[0,1]+roi[1] #y  + plus the shift from the roi
            circle3 = circles[0,2]
    #    #save it to a row
        if circles is None:
            circle1 = "NA"
            circle2 = "NA"
            circle3 = "NA"
        # code for drawing the found circles onto the original video    
        for (x, y, r) in circles:
            #cv2.circle(frame, (x, y), r, (255, 255, 0), 2) #version without drawing roi back on whole image
            #cv2.circle(output[int(roi[1]): int(roi[1]+roi[3]),
            #         int(roi[0]):int(roi[0]+roi[2])], (x, y), r, (255, 255, 0), 2)
            cv2.circle(frame, (int(x+roi[0]), int(y+roi[1])), r, (200, 0, 0), 2)
            
            #print(x,y,r)
    cv2.waitKey(1)
    out.write(frame)
    #cv2.imshow("output", frame)#frame
    i=i+1
    print(i)
    # saving results and used parameters row wise during loop into a dataframe
    new_row = [i+1, roi[0],roi[1], roi[2], roi[3], circle1, circle2,circle3, videofilename, name,
               alpha, beta, dp, minDist, param1, param2, minRadius, maxRadius]
    df.loc[len(df)] = new_row

# cleaning up
out.release()
cap.release()
cv2.destroyAllWindows()

# filename and saving dataframe as cvs file       
filename =  './Output/timeseries/airsacradius_results_' + nameforfiles + '.csv'
df.to_csv(filename, sep = ',')


#smoothing of x,y and r of circles
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.filtfilt.html
#filename =  './Output/timeseries/airsacradius_results_' + nameforfiles + '.csv'
dfraw = pd.read_csv(filename)

# Opens the Video file
cap = cv2.VideoCapture(videofolder+videofilename)
fps = cap.get(cv2.CAP_PROP_FPS) 
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

#outputfile smoothed
out_smoothed = cv2.VideoWriter('./Output/videos/output_smoothed'+videofilename,cv2.VideoWriter_fourcc(*'MP4V'), fps, 
                      (int(frameWidth), int(frameHeight)))

#delete outliers
#how? dynamic? mean +- 2*std of position of centroid in x or y direction? everything under or over that will be deleted
# calculate euclidean distance? to take x and y into account? 

# filter version using buterworth filter (low pass 10 Hz)
#low = 12/(fps/2)
#b, a = scipy.signal.butter(15, low,'low')
        
#smooth_x = scipy.signal.filtfilt(b, a, dfraw['x'], axis=- 1, padtype='odd', padlen=None, method='pad', irlen=None)  
#smooth_y = scipy.signal.filtfilt(b, a, dfraw['y'], axis=- 1, padtype='odd', padlen=None, method='pad', irlen=None)
#smooth_r = scipy.signal.filtfilt(b, a, dfraw['r'], axis=- 1, padtype='odd', padlen=None, method='pad', irlen=None)

#filter version using Savitzky-golay filter
wl = 11 #windowlength for savgol filter, must be odd (reduce the window length it becomes more jittery)
p = 2   # polynomial for savgol filter, must be less than window_length

roi = [dfraw['roi_x1'][0], dfraw['roi_y1'][1], dfraw['roi_x2'][2],dfraw['roi_y2'][3]]

raw_x = dfraw['x']
raw_y = dfraw['y']
raw_r = dfraw['r']

smooth_x = signal.savgol_filter(dfraw['x'], window_length=wl, polyorder=p)
smooth_y = signal.savgol_filter(dfraw['y'], window_length=wl, polyorder=p)
smooth_r = signal.savgol_filter(dfraw['r'], window_length=wl, polyorder=p)

name = dfraw['input_id']
circles_save = list(np.column_stack((smooth_x, smooth_y, smooth_r, name)))
raw_circles = list(np.column_stack((dfraw['x'], dfraw['y'], dfraw['r'])))

column_names = ['x', 'y', 'r', 'inputfile']


df2 = pd.DataFrame(circles_save, columns = column_names)
filename =  './Output/timeseries/airsacradius_results_smoothed' + videofilename + '.csv'
df2.to_csv(filename, sep = ',')

i = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    
    if ret == False:
        break
    output=frame.copy()
    x =int(smooth_x[i])
    y =int(smooth_y[i])
    r =int(smooth_r[i])
    #a,b,c = raw_circles[i]
    rx =int(raw_x[i])
    ry =int(raw_y[i])
    rr =int(raw_r[i])
    #roi
    cv2.rectangle(output,(int(roi[0]), int(roi[1])), (int(roi[0]+roi[2]), int(roi[1]+roi[3])), (0,0,0), 3)
    #unsmoothedcircle
    cv2.circle(output, (rx, ry), rr, (200, 0, 0), 1)
    #smoothed cicle
    cv2.circle(output, (x, y), r, (0,0, 255), 3)
    i= i+1
            
    #cv2.imshow("output", output)
    out_smoothed.write(output)

# cleaning up
out_smoothed.release()
cap.release()
cv2.destroyAllWindows()

#other filter option: savgoy filter
#from scipy.signal import savgol_filter
#def savgol(x, wl=14, p=2):
#    return savgol_filter(x, window_length=wl, polyorder=p)


# to do: filtering is too heavy with (8,0.125) check other combinations, also check whether filter is set correctly for
# our sampling rate --> changed that, fs is now in the function, what does the 8 mean? 
# automatic roi seems to be amiss quite heavily for all videos but 16_20...check again how to improve
# we need to read in roi from the data sheet, otherwise weird things can happen

# Loop to optimize parameters
## looping through manully tracked videos

In [12]:
#version 1, using images ####################loading in the images
#imagefolder = '../TestFrames/testframes_9112022/'
#ims = [f for f in listdir(imagefolder) if isfile(join(imagefolder, f))]
#imlist = []
#for i in ims: #add the image folder name to get the full path
#    imlist.append(imagefolder +i) 
#imgs = [cv2.imread(i) for i in imlist] #laod in the images
#######################################
import random
#version 2, using videos #################### loading in the videos
videofolder = '../Video/for_loop/'
vids = [f for f in listdir(videofolder) if isfile(join(videofolder, f))]
vidlist = []

for i in vids: #add the image folder name to get the full path
    vidlist.append(videofolder +i) 
#videos = [cv2.imread(i) for i in vidlist] #laod in the images
#######################################

# in comments is the first round of parameter optimization we ran
alphas = [2, 2.5, 3]
betas = [25,30,35] 
dilations = [5,6,7] #stayed the same
phase1_medianblurs = [25, 27, 29] #medianblur 1 and 2 are now kept the same, in first otimization blur2 was static at 17
threshs_div_1 = [8,10,12]
threshs_div_2 = [13,15,17]

#static settings
minDist = 10000
minRadius = 5
maxRadius = 270
dp=1

randlistsettings = []
for thresh_div_1 in threshs_div_1:
    for thresh_div_2 in threshs_div_2:
        for alpha in alphas:
            for beta in betas:
                for dilation in dilations:
                    for phase1_medianblur in phase1_medianblurs:
                        values = [thresh_div_1, thresh_div_2, alpha,  beta, dilation, phase1_medianblur]
                        randlistsettings.append(values)
random.shuffle(randlistsettings)

In [13]:
            #rois for 9 manually tracked videos
rois_dict = {'June09_01': [0,0,1503,1056],
             'June09_02': [197,99,940,893],
             'June09_03': [8,96,1156,925],
             'June12_01': [773,302,707,597],
             'June12_02': [655,1,913,1055],
             'June12_03': [276,4,849,1052],
             'June16_02': [617,83,810,838],
             'June16_07': [836,240,858,720],
             'June16_20': [463,122,1154,928]}

In [ ]:
from os.path import exists
#loop through all parameters and videos 
for settings in randlistsettings:
    thresh_div_1=settings[0]
    thresh_div_2=settings[1]
    alpha=settings[2]
    dilation= settings[4]
    beta=settings[3]
    phase1_medianblur=settings[5]             
    for video in vidlist:
        outputcode = 'c1_'+str(thresh_div_1)+'_c2_'+\
                str(thresh_div_2)+'_al_'+str(alpha)+'_b_'+str(beta)+'_dil_'+\
                str(dilation)+'_blur_'+str(phase1_medianblur)
        print('checking for ' + outputcode)
        if not exists('../TestResults/' + outputcode + '_vid_' + os.path.basename(video) + '.csv'):
            print('working on ' + '../TestResults/' + outputcode + '_vid_' + os.path.basename(video) + '.csv')
            #if cannyt1 < cannyt2: #canny 2 needs to be lower

            #set up empty output dataframe
            column_names = ['frame','roi_x1','roi_y1','roi_x2', 'roi_y2', # info on region of interest for repetability
                            'x','y', 'r', 'name',                    # actual results 
                            'alpha', 'beta',                              # values of brightness and contrast pre processing
                            'minDist', 'threh_div1', 'threh_div2', 'dilation', 'phase1_medianblur'] # parameters of hough circle transform 
            df = pd.DataFrame(columns = column_names)
            #outputf = os.path.abspath('./Output/'+outputcode+'.jpeg')
            #if os.path.exists(outputf) == False:
                #cnames = ['example1', 'example2', 'example3', 'example4', 'example5']
                #rnames = ['original_'+str(alpha)+'_'+str(beta), 'blur_' + str(phase1_medianblur), 'dilation_' + str(dilation)]

            #include roi here
            #roi = roidefinition(video = video, height_buffer= 150, width_buffer = 300, dp =1, minRadius = minRadius, maxRadius = maxRadius)   
            
            roi = rois_dict[os.path.basename(video)[0:-4]]
            
                ####################DO ROUTINEand 
            cap = cv2.VideoCapture(video)
            frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
            frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
            fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second
            nameforfiles = i[0:-4]
            imagesprocessed = []
            #for imtoprocess in videos:
            j = 0   
            while(cap.isOpened()):
                    ret, frame = cap.read()
                    j=j+1
                    name = 'framenr_' + str(j) + '_framevid_' +  os.path.basename(video)[0:-4]
                    if ret == False:
                        break
                    ############################detect circles   
                    output=frame.copy()
                    image4, param1, param2 = preprocessing2(image=output)
                        #track demicircles 
                    name = 'framenr_' + str(j) + '_framevid_' + os.path.basename(video[0:-4])
                        # original values: dp = 1, minDist = 10000, param1=1, param2=10, minRadius = 5, maxRadius= 250
                        #submit to 
                    image4 = image4[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]
                    final_im = cv2.normalize(src=image4, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) 
                    circles = cv2.HoughCircles(final_im, cv2.HOUGH_GRADIENT, 
                                                   dp = dp,minDist = minDist,  
                                                   param1 = param1,param2 = param2,
                                                   minRadius = minRadius, maxRadius = maxRadius)   

                    if circles is not None:
                            #circles = np.round(circles[0, 0:1]).astype("int")
                        if circles is not None:
                            circles = np.round(circles[0, 0:1]).astype("int")
                            circle1 = circles[0,0]+roi[0] #x  + plus the shift from the roi
                            circle2 = circles[0,1]+roi[1] #y  + plus the shift from the roi
                            circle3 = circles[0,2]
                        #    #save it to a row
                        if circles is None:
                            circle1 = "NA"
                            circle2 = "NA"
                            circle3 = "NA"
                    new_row = [j, roi[0],roi[1], roi[2], roi[3], circle1, circle2,circle3, name,
                                   alpha, beta, minDist, thresh_div_1, thresh_div_2, str(dilation), str(phase1_medianblur)]
                    df.loc[len(df)] = new_row
            print("processed video")
            filename =  '../TestResults/' + outputcode + '_vid_' + os.path.basename(video[0:-4]) + '.csv'
            df.to_csv(filename, sep = ',')
print('done!')

checking for c1_10_c2_15_al_2.5_b_35_dil_6_blur_25
working on ../TestResults/c1_10_c2_15_al_2.5_b_35_dil_6_blur_25_vid_June09_01.mp4.csv
processed video
checking for c1_10_c2_15_al_2.5_b_35_dil_6_blur_25
working on ../TestResults/c1_10_c2_15_al_2.5_b_35_dil_6_blur_25_vid_June09_02.mp4.csv
processed video
checking for c1_10_c2_15_al_2.5_b_35_dil_6_blur_25
working on ../TestResults/c1_10_c2_15_al_2.5_b_35_dil_6_blur_25_vid_June09_03.mp4.csv
processed video
checking for c1_10_c2_15_al_2.5_b_35_dil_6_blur_25
working on ../TestResults/c1_10_c2_15_al_2.5_b_35_dil_6_blur_25_vid_June12_01.mp4.csv
processed video
checking for c1_10_c2_15_al_2.5_b_35_dil_6_blur_25
working on ../TestResults/c1_10_c2_15_al_2.5_b_35_dil_6_blur_25_vid_June12_02.mp4.csv
processed video
checking for c1_10_c2_15_al_2.5_b_35_dil_6_blur_25
working on ../TestResults/c1_10_c2_15_al_2.5_b_35_dil_6_blur_25_vid_June12_03.mp4.csv
processed video
checking for c1_10_c2_15_al_2.5_b_35_dil_6_blur_25
working on ../TestResults/c1_10

processed video
checking for c1_12_c2_13_al_2_b_25_dil_6_blur_25
working on ../TestResults/c1_12_c2_13_al_2_b_25_dil_6_blur_25_vid_June09_02.mp4.csv
processed video
checking for c1_12_c2_13_al_2_b_25_dil_6_blur_25
working on ../TestResults/c1_12_c2_13_al_2_b_25_dil_6_blur_25_vid_June09_03.mp4.csv
processed video
checking for c1_12_c2_13_al_2_b_25_dil_6_blur_25
working on ../TestResults/c1_12_c2_13_al_2_b_25_dil_6_blur_25_vid_June12_01.mp4.csv
processed video
checking for c1_12_c2_13_al_2_b_25_dil_6_blur_25
working on ../TestResults/c1_12_c2_13_al_2_b_25_dil_6_blur_25_vid_June12_02.mp4.csv
processed video
checking for c1_12_c2_13_al_2_b_25_dil_6_blur_25
working on ../TestResults/c1_12_c2_13_al_2_b_25_dil_6_blur_25_vid_June12_03.mp4.csv
processed video
checking for c1_12_c2_13_al_2_b_25_dil_6_blur_25
working on ../TestResults/c1_12_c2_13_al_2_b_25_dil_6_blur_25_vid_June16_02.mp4.csv
processed video
checking for c1_12_c2_13_al_2_b_25_dil_6_blur_25
working on ../TestResults/c1_12_c2_13_al_

processed video
checking for c1_12_c2_17_al_2.5_b_30_dil_5_blur_29
working on ../TestResults/c1_12_c2_17_al_2.5_b_30_dil_5_blur_29_vid_June12_01.mp4.csv
processed video
checking for c1_12_c2_17_al_2.5_b_30_dil_5_blur_29
working on ../TestResults/c1_12_c2_17_al_2.5_b_30_dil_5_blur_29_vid_June12_02.mp4.csv
processed video
checking for c1_12_c2_17_al_2.5_b_30_dil_5_blur_29
working on ../TestResults/c1_12_c2_17_al_2.5_b_30_dil_5_blur_29_vid_June12_03.mp4.csv
processed video
checking for c1_12_c2_17_al_2.5_b_30_dil_5_blur_29
working on ../TestResults/c1_12_c2_17_al_2.5_b_30_dil_5_blur_29_vid_June16_02.mp4.csv


# Loop through all videos in the video folder